In [1]:
# | default_exp _components.aiokafka_consumer_loop

In [2]:
# | export

import asyncio
from asyncio import iscoroutinefunction  # do not use the version from inspect
from datetime import datetime, timedelta
from os import environ
from typing import *

from fastcore.meta import delegates
import anyio
from anyio.streams.memory import MemoryObjectReceiveStream
import asyncer
from aiokafka import AIOKafkaConsumer
from aiokafka.structs import ConsumerRecord, TopicPartition
from pydantic import BaseModel, Field, HttpUrl, NonNegativeInt

from fast_kafka_api._components.logger import get_logger

In [3]:
from unittest.mock import AsyncMock, MagicMock, Mock, call, patch

from fast_kafka_api._components.logger import supress_timestamps
from fast_kafka_api.testing import (
    create_and_fill_testing_topic,
    nb_safe_seed,
    true_after,
)

In [4]:
seed = nb_safe_seed("_components.aiokafka_consumer_loop")

In [5]:
# | notest
# allows async calls in notebooks

import nest_asyncio

In [6]:
# | notest

nest_asyncio.apply()

In [7]:
# | export

logger = get_logger(__name__)

In [8]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [9]:
# | export

kafka_server_url = (
    environ["KAFKA_HOSTNAME"] if "KAFKA_HOSTNAME" in environ else "localhost"
)

kafka_server_port = environ["KAFKA_PORT"] if "KAFKA_PORT" in environ else "9092"

aiokafka_config = {
    "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}",
}

In [10]:
class MyMessage(BaseModel):
    url: HttpUrl = Field(..., example="http://www.acme.com", description="Url example")
    port: NonNegativeInt = Field(1000)

In [11]:
def create_consumer_record(topic: str, partition: int, msg: BaseModel):
    record = ConsumerRecord(
        topic=topic,
        partition=partition,
        offset=0,
        timestamp=0,
        timestamp_type=0,
        key=None,
        value=msg.json().encode("utf-8") if hasattr(msg, "json") else msg.encode("utf-8"),
        checksum=0,
        serialized_key_size=0,
        serialized_value_size=0,
        headers=[],
    )
    return record

In [12]:
# | export


def _create_safe_callback(
    callback: Callable[[BaseModel], Awaitable[None]]
) -> Callable[[BaseModel], Awaitable[None]]:
    """
    Wraps an async callback into a safe callback that catches any Exception and loggs them as warnings

    Params:
        callback: async callable that will be wrapped into a safe callback

    Returns:
        Wrapped callback into a safe callback that handles exceptions
    """
    async def _safe_callback(
        msg: BaseModel,
        callback: Callable[[BaseModel], Awaitable[None]] = callback,
    ) -> None:
        try:
            #                         logger.debug(f"process_msgs(): awaiting '{callback}({msg})'")
            await callback(msg)
        except Exception as e:
            logger.warning(
                f"_safe_callback(): exception caugth {e.__repr__()} while awaiting '{callback}({msg})'"
            )
    return _safe_callback

In [13]:
# Check if callback is called when wrapped

example_msg = "Example msg"
callback = AsyncMock()
safe_callback = _create_safe_callback(callback)

await safe_callback(f"{example_msg}")

callback.assert_awaited_once_with(f"{example_msg}")

In [14]:
# Check if exception is caught and logged when callback is called and throws an exception

with patch.object(logger, "warning") as mock:
    example_msg = "Example msg"
    exception = Exception("")

    callback = AsyncMock()
    callback.side_effect = exception
    safe_callback = _create_safe_callback(callback)

    await safe_callback(f"{example_msg}")
    
    callback.assert_awaited_once_with(f"{example_msg}")
    mock.assert_called_once_with(
        f"_safe_callback(): exception caugth {exception.__repr__()} while awaiting '{callback}({example_msg})'"
    )

In [15]:
# | export


def _prepare_callback(
    callback: Union[Callable[[BaseModel], None], Callable[[BaseModel], Awaitable[None]]]
) -> Callable[[BaseModel], Awaitable[None]]:
    """
    Prepares a callback to be used in the consumer loop. 
        1. If callback is sync, asyncify it
        2. Wrap the callback into a safe callback for exception handling

    Params:
        callback: async callable that will be prepared for use in consumer

    Returns:
        Prepared callback
    """
    callback: Callable[[BaseModel], Awaitable[None]] = (
        callback if iscoroutinefunction(callback) else asyncer.asyncify(callback)
    )
    return _create_safe_callback(callback)

In [16]:
# Check if callback is called when wrapped

for is_async in [False, True]:
    example_msg = "Example msg"
    callback = AsyncMock() if is_async else Mock()
    prepared_callback = _prepare_callback(callback)

    await prepared_callback(f"{example_msg}")

    callback.assert_called_once_with(f"{example_msg}")

In [17]:
# | export


async def _stream_msgs(msgs: Dict[TopicPartition, bytes], send_stream: anyio.streams.memory.MemoryObjectSendStream[Any]) -> None:
    """
    Decodes and streams the message and topic to the send_stream. 
    
    Params:
        msgs:
        send_stream:
    """
    for topic_partition, topic_msgs in msgs.items():
        topic = topic_partition.topic
        try:
            await send_stream.send(topic_msgs)
        except Exception as e:
            logger.warning(
                f"_stream_msgs(): Unexpected exception '{e.__repr__()}' caught and ignored for topic='{topic_partition.topic}', partition='{topic_partition.partition}' and messages: {topic_msgs}"
            )
            
def _decode_streamed_msgs(msgs: List[ConsumerRecord], msg_type: BaseModel) -> List[BaseModel]:
    decoded_msgs = [msg_type.parse_raw(msg.value.decode("utf-8")) for msg in msgs]
    return decoded_msgs

In [18]:
# Sanity check: one msg, one topic

with patch("anyio.streams.memory.MemoryObjectSendStream.send") as mock:
    send_stream, receive_stream = anyio.create_memory_object_stream()

    topic = "topic_0"
    partition = 0
    topic_part_0_0 = TopicPartition(topic, partition)
    msg = MyMessage(url="http://www.acme.com", port=22)
    record = create_consumer_record(topic=topic, partition=partition, msg=msg)

    await _stream_msgs(
        msgs={topic_part_0_0: [record]},
        send_stream=send_stream,
    )

    mock.assert_called_once()
    mock.assert_has_calls([call([record])])
    
#     received_msgs = mock.call_args[0][0]
#     decoded_msgs = _decode_streamed_msgs(received_msgs, MyMessage)
#     assert [msg] == decoded_msgs

In [19]:
# Check different topics

# Two msg, two topics, send called twice with each topic

with patch("anyio.streams.memory.MemoryObjectSendStream.send") as mock:
    send_stream, receive_stream = anyio.create_memory_object_stream()

    topic_partitions = [("topic_0", 0), ("topic_1", 0)]

    msg = MyMessage(url="http://www.acme.com", port=22)
    msgs = {
        TopicPartition(topic, partition): [
            create_consumer_record(topic=topic, partition=partition, msg=msg)
        ]
        for topic, partition in topic_partitions
    }

    await _stream_msgs(
        msgs=msgs,
        send_stream=send_stream,
    )

    assert mock.call_count == 2

    mock.assert_has_calls([call(msg) for msg in msgs.values()])

In [20]:
# Check multiple msgs in same topic

# Two msg, one topic, send called twice for same topic

with patch("anyio.streams.memory.MemoryObjectSendStream.send") as mock:
    send_stream, receive_stream = anyio.create_memory_object_stream()

    topic_partitions = [("topic_0", 0)]

    msg = MyMessage(url="http://www.acme.com", port=22)
    record = create_consumer_record(topic=topic, partition=partition, msg=msg)

    msgs = {
        TopicPartition(topic, partition): [
            create_consumer_record(topic=topic, partition=partition, msg=msg),
            create_consumer_record(topic=topic, partition=partition, msg=msg),
        ]
        for topic, partition in topic_partitions
    }

    await _stream_msgs(
        msgs=msgs,
        send_stream=send_stream,
    )

    mock.assert_has_calls([call(msg) for msg in msgs.values()])

In [21]:
# Check multiple partitions

# Two msg, one topic, differenct partitions, send called twice for same topic

with patch("anyio.streams.memory.MemoryObjectSendStream.send") as mock:
    send_stream, receive_stream = anyio.create_memory_object_stream()

    topic_partitions = [("topic_0", 0), ("topic_0", 1)]

    msg = MyMessage(url="http://www.acme.com", port=22)
    msgs = {
        TopicPartition(topic, partition): [
            create_consumer_record(topic=topic, partition=partition, msg=msg)
        ]
        for topic, partition in topic_partitions
    }
    record = create_consumer_record(topic=topic, partition=partition, msg=msg)

    await _stream_msgs(
        msgs=msgs,
        send_stream=send_stream,
    )

    mock.assert_has_calls([call(msg) for msg in msgs.values()])

In [22]:
# | export


@delegates(AIOKafkaConsumer.getmany)
async def _aiokafka_consumer_loop(  # type: ignore
    consumer: AIOKafkaConsumer,
    *,
    topic: str,
    callback: Callable[[BaseModel], Union[None, Awaitable[None]]],
    max_buffer_size: int = 100_000,
    msg_type: Type[BaseModel],
    is_shutting_down_f: Callable[[], bool],
    **kwargs,
) -> None:
    """
    Consumer loop for infinite pooling of the AIOKafka consumer for new messages. Calls consumer.getmany()
    and after the consumer return messages or times out, messages are decoded and streamed to defined callback.

    Params:
        callbacks: Dict of callbacks mapped to their respective topics
        timeout_ms: Time to timeut the getmany request by the consumer
        max_buffer_size: Maximum number of unconsumed messages in the callback buffer
        msg_types: Dict of message types mapped to their respective topics
        is_shutting_down_f: Function for controlling the shutdown of consumer loop
    """

    prepared_callback = _prepare_callback(callback)

    async def process_message_callback(
        receive_stream: MemoryObjectReceiveStream[Any],
        callback=prepared_callback,
        msg_type=msg_type,
        topic=topic,
    ) -> None:
        async with receive_stream:
            try:
                # todo: iteration can be factored out as a function
                async for records_per_topic in receive_stream:
                    for records in records_per_topic:
                        for record in records:
                            try:
                                msg = record.value
                                decoded_msg = msg_type.parse_raw(msg.decode("utf-8"))
                                await callback(decoded_msg)
                            except Exception as e:
                                logger.warning(
                                    f"process_message_callback(): Unexpected exception '{e.__repr__()}' caught and ignored for topic='{topic}' and message: {msg}"
                                )
            except Exception as e:
                logger.warning(
                    f"process_message_callback(): Unexpected exception '{e.__repr__()}' caught and ignored for topic='{topic}' and message: {msg}"
                )

    send_stream, receive_stream = anyio.create_memory_object_stream(
        max_buffer_size=max_buffer_size
    )

    async with anyio.create_task_group() as tg:
        tg.start_soon(process_message_callback, receive_stream)
        async with send_stream:
            while not is_shutting_down_f():
                msgs = await consumer.getmany(**kwargs)
                try:
#                     await _stream_msgs(msgs, send_stream)
#                     print(f"{msgs=}")
                    await send_stream.send(msgs.values())
                except Exception as e:
                    logger.warning(
                        f"_aiokafka_consumer_loop(): Unexpected exception '{e}' caught and ignored for messages: {msgs}"
                    )

In [23]:
def is_shutting_down_f(mock_func: Mock, num_calls: int = 1) -> Callable[[], bool]:
    def _is_shutting_down_f():
        return mock_func.call_count == num_calls

    return _is_shutting_down_f

In [24]:
topic = "topic_0"
partition = 0
msg = MyMessage(url="http://www.acme.com", port=22)
record = create_consumer_record(topic=topic, partition=partition, msg=msg)

mock_consumer = MagicMock()
msgs = {TopicPartition(topic, 0): [record]}

f = asyncio.Future()
f.set_result(msgs)
mock_consumer.configure_mock(**{"getmany.return_value": f})
mock_callback = Mock()


for is_async in [True, False]:
    await _aiokafka_consumer_loop(
        consumer=mock_consumer,
        topic=topic,
        max_buffer_size=100,
        timeout_ms=10,
        callback=asyncer.asyncify(mock_callback) if is_async else mock_callback,
        msg_type=MyMessage,
        is_shutting_down_f=is_shutting_down_f(mock_consumer.getmany),
    )

    assert mock_consumer.getmany.call_count == 1
    mock_callback.assert_called_once_with(msg)

In [25]:
# Sanity check: exception in callback recovery
# Two msg, one topic, process_f called twice even tough it throws

topic = "topic_0"
partition = 0
msg = MyMessage(url="http://www.acme.com", port=22)
record = create_consumer_record(topic=topic, partition=partition, msg=msg)

num_msgs = 2

mock_consumer = MagicMock()
msgs = {TopicPartition(topic, 0): [record, record]}

f = asyncio.Future()
f.set_result(msgs)

mock_consumer.configure_mock(**{"getmany.return_value": f})
mock_callback = Mock()

exception = Exception("")
mock_callback.side_effect = exception

for is_async in [True, False]:
    await _aiokafka_consumer_loop(
        consumer=mock_consumer,
        topic=topic,
        max_buffer_size=100,
        timeout_ms=1,
        callback= asyncer.asyncify(mock_callback) if is_async else mock_callback,
        msg_type=MyMessage,
        is_shutting_down_f=is_shutting_down_f(mock_consumer.getmany, num_calls=1),
    )

    assert mock_callback.call_count == num_msgs, mock_callback.call_count
    mock_callback.assert_has_calls([call(msg), call(msg)])

print("ok")

[WARNING] __main__: _safe_callback(): exception caugth Exception('') while awaiting '<function asyncify.<locals>.wrapper at 0x7f8ead457250>(url=HttpUrl('http://www.acme.com', ) port=22)'
[WARNING] __main__: _safe_callback(): exception caugth Exception('') while awaiting '<function asyncify.<locals>.wrapper at 0x7f8ead457250>(url=HttpUrl('http://www.acme.com', ) port=22)'
ok


In [26]:
# Sanity check: malformed msgs
# One msg of wrong type, two normal msg, one topic, process_f called twice

topic = "topic_0"
partition = 0
msg = MyMessage(url="http://www.acme.com", port=22)
correct_record = create_consumer_record(topic=topic, partition=partition, msg=msg)
faulty_record = create_consumer_record(topic=topic, partition=partition, msg="Wrong!")

mock_consumer = MagicMock()
msgs = {TopicPartition(topic, 0): [faulty_record, correct_record, correct_record]}

mock_consumer.configure_mock(**{"getmany.return_value": f})
mock_callback = Mock()

exception = Exception("")
callback.side_effect = exception

for is_async in [True, False]:
    await _aiokafka_consumer_loop(
        consumer=mock_consumer,
        topic=topic,
        max_buffer_size=100,
        timeout_ms=10,
        callback=asyncer.asyncify(mock_callback) if is_async else mock_callback,
        msg_type=MyMessage,
        is_shutting_down_f=is_shutting_down_f(mock_consumer.getmany),
    )

    assert mock_consumer.getmany.call_count == 1
    mock_callback.assert_has_calls([call(msg), call(msg)])

print("ok")

ok


In [27]:
#| export

def sanitize_kafka_config(**kwargs):
    """Sanitize Kafka config"""
    return {k: "*"*len(v) if "pass" in k.lower() else v for k, v in kwargs.items()}

In [28]:
kwargs = {'bootstrap_servers': 'whatever.cloud:9092',
 'auto_offset_reset': 'earliest',
 'security_protocol': 'SASL_SSL',
 'sasl_mechanism': 'PLAIN',
 'sasl_plain_username': 'username',
 'sasl_plain_password': 'password',
 'ssl_context': "something"}

assert sanitize_kafka_config(**kwargs)["sasl_plain_password"] == '********'

In [29]:
# | export


@delegates(AIOKafkaConsumer)
@delegates(_aiokafka_consumer_loop, keep=True)
async def aiokafka_consumer_loop(  # type: ignore
    topic: str,
    *,
    timeout_ms: int = 100,
    max_buffer_size: int = 100_000,
    callback: Dict[str, Callable[[BaseModel], Union[None, Awaitable[None]]]],
    msg_type: Dict[str, Type[BaseModel]],
    is_shutting_down_f: Callable[[], bool],
    **kwargs,
) -> None:
    """Consumer loop for infinite pooling of the AIOKafka consumer for new messages. Creates and starts AIOKafkaConsumer
    and runs _aio_kafka_consumer loop fo infinite poling of the consumer for new messages.

    Args:
        topic: name of the topic to subscribe to
        callback: callback function to be called after decoding and parsing a consumed message
        timeout_ms: Time to timeut the getmany request by the consumer
        max_buffer_size: Maximum number of unconsumed messages in the callback buffer
        msg_type: Type with `parse_json` method used for parsing a decoded message
        is_shutting_down_f: Function for controlling the shutdown of consumer loop
    """
    logger.info(f"aiokafka_consumer_loop() starting...")
    try:
        consumer = AIOKafkaConsumer(
            **kwargs,
        )
        logger.info(
            f"aiokafka_consumer_loop(): Consumer created using the following parameters: {sanitize_kafka_config(**kwargs)}"
        )

        await consumer.start()
        logger.info("aiokafka_consumer_loop(): Consumer started.")
        consumer.subscribe([topic])
        logger.info("aiokafka_consumer_loop(): Consumer subscribed.")

        try:
            await _aiokafka_consumer_loop(
                consumer=consumer,
                topic=topic,
                max_buffer_size=max_buffer_size,
                timeout_ms=timeout_ms,
                callback=callback,
                msg_type=msg_type,
                is_shutting_down_f=is_shutting_down_f,
            )
        finally:
            await consumer.stop()
            logger.info(f"aiokafka_consumer_loop(): Consumer stopped.")
            logger.info(f"aiokafka_consumer_loop() finished.")
    except Exception as e:
        logger.error(
            f"aiokafka_consumer_loop(): unexpected exception raised: '{e.__repr__()}'"
        )
        raise e

In [33]:
msgs_sent = 9178
msgs = [
    MyMessage(url="http://www.ai.com", port=port).json().encode("utf-8")
    for port in range(msgs_sent)
]
msgs_received = 0


async def count_msg(msg: MyMessage):
    global msgs_received
    msgs_received = msgs_received + 1
    if msgs_received % 1000 == 0:
        logger.info(f"{msgs_received=}")


async with create_and_fill_testing_topic(
    topic_prefix="my_topic_aiokafka_consumer_loop_",
    msgs=msgs,
    seed=seed(1),
    **aiokafka_config,
) as topic:
    await aiokafka_consumer_loop(
        topic=topic,
        auto_offset_reset="earliest",
        callback=count_msg,
        msg_type=MyMessage,
        is_shutting_down_f=true_after(2),
        **aiokafka_config,
    )

assert msgs_sent == msgs_received, f"{msgs_sent} != {msgs_received}"

[INFO] fast_kafka_api.helpers: create_missing_topics(['my_topic_aiokafka_consumer_loop_5696213874']): new_topics = [NewTopic(topic=my_topic_aiokafka_consumer_loop_5696213874,num_partitions=2)]


producing to 'my_topic_aiokafka_consumer_loop_5696213874':   0%|          | 0/9178 [00:00<?, ?it/s]

[INFO] __main__: aiokafka_consumer_loop() starting...
[INFO] __main__: aiokafka_consumer_loop(): Consumer created using the following parameters: {'auto_offset_reset': 'earliest', 'group_id': 'some_id', 'bootstrap_servers': 'tvrtko-fast-kafka-api-kafka-1:9092'}
[INFO] __main__: aiokafka_consumer_loop(): Consumer started.
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_topic_aiokafka_consumer_loop_5696213874'})
[INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'my_topic_aiokafka_consumer_loop_5696213874'}
[INFO] __main__: aiokafka_consumer_loop(): Consumer subscribed.
[INFO] aiokafka.consumer.group_coordinator: Discovered coordinator 1001 for group some_id
[INFO] aiokafka.consumer.group_coordinator: Revoking previously assigned partitions set() for group some_id
[INFO] aiokafka.consumer.group_coordinator: (Re-)joining group some_id
[INFO] aiokafka.consumer.group_coordinator: Joined group 'some_id' (generation 1) with member_id aiokafk

In [32]:
# | notest

msgs_sent = 100_000
msgs = [
    MyMessage(url="http://www.ai.com", port=port).json().encode("utf-8")
    for port in range(msgs_sent)
]
msgs_received = 0


async def count_msg(msg: MyMessage):
    global msgs_received
    msgs_received = msgs_received + 1
    if msgs_received % 1000 == 0:
        logger.info(f"{msgs_received=}")


def _is_shutting_down_f():
    return msgs_received == msgs_sent


async with create_and_fill_testing_topic(
    topic_prefix="my_topic_aiokafka_consumer_loop_",
    msgs=msgs,
    seed=seed(3),
    **aiokafka_config,
) as topic:
    start = datetime.now()
    await aiokafka_consumer_loop(
        topic=topic,
        auto_offset_reset="earliest",
        callback=count_msg,
        msg_type=MyMessage,
        is_shutting_down_f=_is_shutting_down_f,
        **aiokafka_config
    )
    t = (datetime.now() - start) / timedelta(seconds=1)
    thrp = msgs_received / t

    print(f"Messages processed: {msgs_received:,d}")
    print(f"Time              : {t:.2f} s")
    print(f"Throughput.       : {thrp:,.0f} msg/s")

[INFO] fast_kafka_api.helpers: create_missing_topics(['my_topic_aiokafka_consumer_loop_5168585847']): new_topics = [NewTopic(topic=my_topic_aiokafka_consumer_loop_5168585847,num_partitions=2)]


producing to 'my_topic_aiokafka_consumer_loop_5168585847':   0%|          | 0/100000 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'dict'